## Numpy를 사용하여 RNN 동작 이해

In [29]:
import numpy as np

# RNN이 몇 번 펼쳐질 것인가? => 문장의 길이
timesteps = 10  # 문장의 길이 t

# RNN의 입력. 일반적으로는 단어의 벡터 차원
input_size = 4  # 단어의 차원 D

# RNN cell에서 hidden unit의 갯수 (cell 용량)
hidden_size = 8

In [30]:
# RNN 입력 데이터 (projection layer)
inputs = np.random.random((timesteps, input_size))  # D x t

# hidden state
hidden_state_t = np.zeros((hidden_size,))  # h_0 = 0

In [31]:
inputs

array([[0.4740404 , 0.70393955, 0.07639192, 0.07274193],
       [0.36624767, 0.60855802, 0.97340462, 0.33911153],
       [0.09112802, 0.53190302, 0.62253596, 0.15253293],
       [0.96735825, 0.42700491, 0.00766842, 0.82660868],
       [0.938348  , 0.82702749, 0.18642995, 0.27837966],
       [0.26104421, 0.43675511, 0.51025156, 0.15675488],
       [0.43244996, 0.96730805, 0.75777186, 0.46823261],
       [0.48359134, 0.50054584, 0.97773736, 0.50846267],
       [0.72626825, 0.96900907, 0.26672834, 0.64730058],
       [0.5306335 , 0.45031866, 0.14121373, 0.79564098]])

In [32]:
print(hidden_state_t)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [33]:
# RNN cell 뉴런의 가중치 설정
# 1. 입력 x_t 와 대응되는 가중치 (D x hidden_size)
# 2. 이전 시점의 상태인 h_(t-1) 에 대응되는 가중치 (hidden_size x hidden_size)

W_x = np.random.random((hidden_size, input_size))
W_h = np.random.random((hidden_size, hidden_size))
b = np.random.random((hidden_size,))

$$
h_t = tanh(W_xx_t + W_hh_{t-1} + b)
$$

In [34]:
total_hidden_states = []

# RNN 작동

# 단어 벡터 하나씩 순서대로 꺼낸다
for input_t in inputs:
    # output_t 가 실제로는 h_t 의 역할 (현 시점의 hidden state)
    output_t = np.tanh(W_x @ input_t + W_h @ hidden_state_t + b)

    # 각 시점의 은닉 상태의 값을 계속해서 기록
    total_hidden_states.append(list(output_t))

    hidden_state_t = output_t

# 출력 시 값을 깔끔하게 만들어줌
total_hidden_states = np.stack(total_hidden_states, axis=0)

print(total_hidden_states)

[[0.57233133 0.83320468 0.62405135 0.5563321  0.73192675 0.75505076
  0.76346329 0.60714161]
 [0.99970929 0.99934681 0.9984037  0.99568118 0.99852231 0.99908636
  0.99948534 0.99836627]
 [0.99996845 0.99974602 0.99983944 0.99857436 0.99958013 0.99974239
  0.99980824 0.99979578]
 [0.99998353 0.99992415 0.99991764 0.99967546 0.99969424 0.99987357
  0.99983488 0.99987852]
 [0.99997063 0.99985376 0.9999276  0.99925842 0.99982861 0.99987391
  0.99987287 0.99986328]
 [0.99996383 0.99974692 0.99985283 0.99863859 0.99954205 0.99976828
  0.99978305 0.99979495]
 [0.99998859 0.99992327 0.99991825 0.99944892 0.99989465 0.99988418
  0.99994144 0.99990164]
 [0.99998844 0.99994257 0.9999174  0.99947613 0.99985617 0.99991842
  0.99993841 0.99990751]
 [0.99998721 0.99991942 0.99992409 0.9995987  0.99986405 0.99986662
  0.99991239 0.99989391]
 [0.99998451 0.99989823 0.99988029 0.9995821  0.99958533 0.9998003
  0.99980867 0.99985471]]


## PyTorch 를 이용한 RNN

In [35]:
import torch
import torch.nn as nn

In [36]:
input_size = 5  # 입력되는 단어 벡터의 차원
hidden_size = 8  # 셀의 용량

`[I, am, a , Student]`라는 문장이 들어간다고 가정


In [37]:
# (batch_size, timesteps, input_size) ==> (배치 크기, 문장 길이, 단어 벡터 크기)
inputs = torch.Tensor(1, 4, input_size)
print(inputs)

tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])


In [38]:
# (데이터의 개수/ 배치 크기, 문장의 길이, 단어 벡터의 차원)
inputs.shape

torch.Size([1, 4, 5])

In [41]:
# RNN 레이어에 필요한 것 => 입력 데이터 차원, 뉴런 갯수
cell = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)

In [42]:
outputs, hidden = cell(inputs)

In [45]:
# 모든 timestep 각각의 hidden state
outputs

tensor([[[-0.0136,  0.1741, -0.0791,  0.4193, -0.4857,  0.0182, -0.4776,
           0.4980],
         [ 0.1125,  0.1758, -0.1855,  0.5135, -0.4631, -0.0884, -0.4938,
           0.5450],
         [ 0.1371,  0.0888, -0.1683,  0.5545, -0.4568, -0.0620, -0.4910,
           0.5196],
         [ 0.1514,  0.0985, -0.1528,  0.5467, -0.4385, -0.0661, -0.5066,
           0.4927]]], grad_fn=<TransposeBackward1>)

In [46]:
# 제일 마지막 timestep의 hidden state
hidden

tensor([[[ 0.1514,  0.0985, -0.1528,  0.5467, -0.4385, -0.0661, -0.5066,
           0.4927]]], grad_fn=<StackBackward0>)

## many to one, many to many

- Many to One은 여러 timestep을 입력 받아 하나의 결과를 낸다.
- Many to Many는 여러 timestep을 입력 받아 여러 결과를 낸다.

## Deep RNN (Deep Recurrent Neural Network)
RNN 층이 여러 겹으로 쌓여있는 경우

In [47]:
# ( batch_size, timestep, input_size )
inputs = torch.Tensor(1, 4, 5)

In [49]:
cell = nn.RNN(
    input_size=5, hidden_size=8, batch_first=True, num_layers=2  # RNN 층이 몇개인지
)

outputs, hidden = cell(inputs)

In [52]:
# outputs는 모든 timestep 각각의 hidden state 이므로 층이 몇 층이던 동일
outputs.shape

torch.Size([1, 4, 8])

In [53]:
# 각 층의 마지막 timestep (층 수, batch size, hidden state)
hidden.shape

torch.Size([2, 1, 8])

## Bi-RNN (Bidirectional RNN)

In [54]:
inputs = torch.Tensor(1, 4, 5)

In [56]:
cell = nn.RNN(input_size=5, hidden_size=8, batch_first=True, bidirectional=True)
outputs, hidden = cell(inputs)

In [57]:
# (1, 4, 16) 인 이유 : 순방향, 역방향 hidden state 각 8개
outputs.shape

torch.Size([1, 4, 16])

In [61]:
# hidden[0] : 순방향 hidden state
# hidden[1] : 1 번쨰가 역방향 hidden state

hidden.shape

torch.Size([2, 1, 8])